In [ ]:


User
用torch写线性回归
ChatGPT
在PyTorch中实现线性回归模型通常涉及以下几个步骤：定义模型、选择损失函数、选择优化器，并进行训练。下面是一个简单的线性回归模型的完整实现示例：

导入必要的库：
torch：基本的PyTorch库。
torch.nn：用于构建神经网络。
torch.optim：包含各种优化算法。
定义数据：
这里我们使用一些随机生成的数据作为示例。
定义线性模型：
使用torch.nn.Linear创建一个线性模型。
定义损失函数和优化器：
使用均方误差作为损失函数。
使用SGD（随机梯度下降）作为优化器。
训练模型：
在循环中执行前向传播、计算损失、执行后向传播（计算梯度）和优化器步骤（更新权重）。
下面是这些步骤的具体代码实现：

python
Copy code
import torch
import torch.nn as nn
import torch.optim as optim

# 生成一些数据点
x_data = torch.randn(100, 1)  # 100个数据点
y_data = x_data * 2 + 3       # 线性关系 y = 2x + 3

# 定义模型
model = nn.Linear(1, 1)  # 输入特征数为1，输出特征数也为1

# 定义损失函数
criterion = nn.MSELoss()

# 定义优化器
optimizer = optim.SGD(model.parameters(), lr=0.01)

# 训练模型
num_epochs = 100  # 训练100轮
for epoch in range(num_epochs):
    # 零梯度
    optimizer.zero_grad()

    # 前向传播
    outputs = model(x_data)

    # 计算损失
    loss = criterion(outputs, y_data)

    # 反向传播
    loss.backward()

    # 更新参数
    optimizer.step()

    # 打印训练信息
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# 查看训练后的模型参数
params = list(model.parameters())
print('Weight:', params[0].item())
print('Bias:', params[1].item())

In [4]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import time

# 定义神经网络模型
device = torch.device(device='cuda' if torch.cuda.is_available() else 'cpu')

class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes, ini_bias):
        super(NeuralNet, self).__init__()
        linear_with_bias = nn.Linear(input_size, hidden_size, bias=True)
        nn.init.constant_(linear_with_bias.bias, ini_bias)
        nn.init.kaiming_normal_(linear_with_bias.weight, mode='fan_in')
        self.layers = nn.ModuleList([linear_with_bias])
        for _ in range(num_layers-1):
            hidden_with_bias = nn.Linear(hidden_size, hidden_size, bias=True)
            nn.init.constant_(hidden_with_bias.bias, ini_bias)
            nn.init.kaiming_normal_(linear_with_bias.weight, mode='fan_in')
            self.layers.append(hidden_with_bias)

        self.layers.append(nn.Linear(hidden_size, num_classes))
        self.relu = nn.ReLU()
        self.batchnorm = nn.BatchNorm1d(hidden_size)

    def forward(self, x):
        for layer in self.layers[:-1]:
            x = self.batchnorm(self.relu(layer(x)))
        x = self.layers[-1](x)
        return x


# 超参数设置
input_size = 784  # MNIST 图像大小为 28x28，展平后为 784
hidden_size = 1024  # 隐藏层大小
num_layers = 6  # 线性层数量
num_classes = 10  # 类别数
num_epochs = 30
batch_size = 200
learning_rate = 5e-3
ini_bias = -0

# 加载 MNIST 数据集
train_dataset = torchvision.datasets.MNIST(
    root='./data', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = torchvision.datasets.MNIST(
    root='./data', train=False, transform=transforms.ToTensor())

print(len(train_dataset))

# 数据加载器
train_loader = DataLoader(dataset=train_dataset,
                          batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset,
                         batch_size=batch_size, shuffle=False)

# 定义模型、损失函数和优化器
model = NeuralNet(input_size, hidden_size, num_layers, num_classes, ini_bias)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

print(model)

# 训练模型
total_step = len(train_loader)
layer_outputs = [[] for _ in range(num_layers)]

time_converge = time.time()

for epoch in range(num_epochs):

    for i, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)
        images = images.reshape(-1, 28*28)

        # 前向传播和计算损失
        outputs = model(images)
        loss = criterion(outputs, labels)

        # 反向传播和优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print(
                f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{total_step}], Loss: {loss.item()}')

        for j in range(num_layers):
            images_ac = torch.relu(model.layers[j](images))
            batch_norm_layer = nn.BatchNorm1d(hidden_size).to(device)
            images = batch_norm_layer(images_ac)
            zero_count = (images_ac == 0).sum().item()  # 激活函数后为0的个数
            total_count = images.numel()  # 总元素数
            zero_ratio = zero_count / total_count  # 计算比例
            layer_outputs[j].append(zero_ratio)

time_converge = time.time() - time_converge
print(f"Time to converge: {time_converge} seconds")
# 可视化结果
iter = range(len(layer_outputs[0]))
for row in range(len(layer_outputs)):
    plt.plot(iter, layer_outputs[row][:], label=f"Layer {row}")

plt.xlabel("Iterations")
plt.ylabel("Sparsity")
plt.title("Sparsity of Each Layer over Iterations")
plt.legend()
plt.show()


# 测试模型
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28)
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: {} %'.format(
        100 * correct / total))

60000
NeuralNet(
  (layers): ModuleList(
    (0): Linear(in_features=784, out_features=1024, bias=True)
    (1-5): 5 x Linear(in_features=1024, out_features=1024, bias=True)
    (6): Linear(in_features=1024, out_features=10, bias=True)
  )
  (relu): ReLU()
  (batchnorm): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)
Epoch [1/30], Step [100/300], Loss: 0.37936538457870483
Epoch [1/30], Step [200/300], Loss: 0.3041316270828247
Epoch [1/30], Step [300/300], Loss: 0.1851465404033661
Epoch [2/30], Step [100/300], Loss: 0.09840123355388641
Epoch [2/30], Step [200/300], Loss: 0.1997208148241043
Epoch [2/30], Step [300/300], Loss: 0.09050767868757248
Epoch [3/30], Step [100/300], Loss: 0.13908204436302185
Epoch [3/30], Step [200/300], Loss: 0.11633522063493729
Epoch [3/30], Step [300/300], Loss: 0.0668475553393364
Epoch [4/30], Step [100/300], Loss: 0.05965627729892731
Epoch [4/30], Step [200/300], Loss: 0.053515106439590454
Epoch [4/30], Step [300/300], L

KeyboardInterrupt: 